In [16]:
''' read the file line by line to understanding how the half edge load 
an off file and to save the half edge mesh as an obj file.

author:
    zhangsihao yang
'''
data_path = 'tests/data/bunny.off'
# data_path = 'output.off'


import mesh
import math
from mesh import Halfedge
from mesh import Vertex
# HalfedgeMesh
mesh = mesh.HalfedgeMesh(data_path)
new_vertices = set()

# Assuming Vertex class and other related structures are defined in mesh.py
def print_all_facets_vertices(mesh):
    for facet in mesh.facets:
        vertices = set()
        start_halfedge = facet.halfedge
        current_halfedge = start_halfedge

        while True:
            vertices.add(current_halfedge.vertex)
            current_halfedge = current_halfedge.next
            if current_halfedge == start_halfedge:
                break

        
        vertex_indices = [v.index for v in vertices]
        print(f"Vertices in facet {facet.index}: {vertex_indices}")

def generate_vertex_at_midpoint(mesh, v0, v1, v2, v3):
    new_x = (3/8) * (v0.x + v2.x) + (1/8) * (v1.x + v3.x)
    new_y = (3/8) * (v0.y + v2.y) + (1/8) * (v1.y + v3.y)
    new_z = (3/8) * (v0.z + v2.z) + (1/8) * (v1.z + v3.z)

    new_vertex = Vertex(
        x=new_x,
        y=new_y,
        z=new_z,
        index=len(mesh.vertices),
    )
    mesh.vertices.append(new_vertex)
    new_vertices.add(new_vertex.index)
    return new_vertex

def shared_edge_new_vertex(mesh):
    printed_edges = set()
    new_vertex_prop = []
    for shared_halfedge in mesh.halfedges:
        if shared_halfedge.opposite:
            edge = tuple(sorted([shared_halfedge.vertex.index, shared_halfedge.opposite.vertex.index]))
            if edge not in printed_edges:
                printed_edges.add(edge)
                v0 = shared_halfedge.vertex.index
                v2 = shared_halfedge.opposite.vertex.index
                f1 = shared_halfedge.facet.index
                f2 = shared_halfedge.opposite.facet.index
                # print(f"Shared edge between vertices {v0} and {v2} is shared by facets {f1} and {f2}")
                
                # Get the four vertices of the two facets
                f1_vertices = [
                    shared_halfedge.vertex,
                    shared_halfedge.next.vertex,
                    shared_halfedge.next.next.vertex
                ]
                f2_vertices = [
                    shared_halfedge.opposite.vertex,
                    shared_halfedge.opposite.next.vertex,
                    shared_halfedge.opposite.next.next.vertex
                ]

                # Combine and remove duplicates
                all_vertices = list(set(f1_vertices + f2_vertices))
                
                # Print the unique vertices
                # print(f"Facet {f1} and Facet {f2} unique vertices: {[v.index for v in all_vertices]}")

                # Ensure we have exactly four unique vertices
                if len(all_vertices) == 4:
                    # Ensure v0 and v2 are the vertices of the shared edge
                    v0 = shared_halfedge.vertex
                    v2 = shared_halfedge.opposite.vertex

                    # The other two vertices
                    remaining_vertices = [v for v in all_vertices if v != v0 and v != v2]
                    v1, v3 = remaining_vertices
                    
                    new_vertex = generate_vertex_at_midpoint(mesh, v0, v1, v2, v3)

                    # Store the new vertex properties
                    new_vertex_prop.append({
                        'vertex': new_vertex,
                        'halfedge': shared_halfedge,
                        'facet': shared_halfedge.facet,
                        'opposite_facet':shared_halfedge.opposite.facet,
                        'start': shared_halfedge.vertex.index,
                        'end': shared_halfedge.opposite.vertex.index
                    })

                    # for prop in new_vertex_prop:
                    #    print(f"Shared new vertex at ({prop['vertex'].x}, {prop['vertex'].y}, {prop['vertex'].z}, start {prop['start']}, end {prop['end']}) on halfedge {prop['halfedge'].index} in facet {prop['facet'].index}")
    return new_vertex_prop

def generate_vertex_at_midpoint_non_shared(mesh, v1, v2):
    new_x = (1/2) * (v1.x + v2.x)
    new_y = (1/2) * (v1.y + v2.y)
    new_z = (1/2) * (v1.z + v2.z)

    new_vertex = Vertex(
        x=new_x,
        y=new_y,
        z=new_z,
        index=len(mesh.vertices),
    )
    mesh.vertices.append(new_vertex)
    new_vertices.add(new_vertex.index)
    return new_vertex

def non_shared_edge_new_vertex(mesh):
    printed_edges = set()
    new_vertex_prop = []
    
    for halfedge in mesh.halfedges:
        if not halfedge.opposite:
            edge = tuple(sorted([halfedge.vertex.index, halfedge.next.vertex.index]))
            if edge not in printed_edges:
                printed_edges.add(edge)
                v1 = halfedge.vertex
                v2 = halfedge.prev.vertex
                # print(f"Non-shared edge between vertices {v1.index} and {v2.index}")
                
                # Generate the new vertex at the midpoint
                new_vertex = generate_vertex_at_midpoint_non_shared(mesh, v1, v2)

                # Store the new vertex properties
                new_vertex_prop.append({
                    'vertex': new_vertex,
                    'halfedge': halfedge,
                    'facet': halfedge.facet,
                    'start': halfedge.vertex.index
                })
    # Print new vertex properties
    # for prop in new_vertex_prop:
    #     print(f"Non-shared new vertex at ({prop['vertex'].x}, {prop['vertex'].y}, {prop['vertex'].z}) on halfedge {prop['halfedge'].index} in facet {prop['facet'].index}")

    return new_vertex_prop

def calculate_beta(n):
    if n == 3:
        return 3 / 16
    else:
        return (1 / n) * (5 / 8 - (3 / 8 + 1 / 4 * math.cos(2 * math.pi / n)) ** 2)

def update_old_vertex_position(vertex):
    adjacent_vertices = []
    halfedge = vertex.halfedge
    start = halfedge

    if halfedge.opposite:
        adjacent_vertices = []
        halfedge = vertex.halfedge
        start = halfedge

        while True:
            if halfedge.opposite:  # Ensure opposite is not None
                adjacent_vertices.append(halfedge.opposite.vertex)
                halfedge = halfedge.opposite.prev
            else:
                break
            if halfedge == start:
                break

        n = len(adjacent_vertices)
        beta = calculate_beta(n)

        new_x = (1 - n * beta) * vertex.x + beta * sum(v.x for v in adjacent_vertices)
        new_y = (1 - n * beta) * vertex.y + beta * sum(v.y for v in adjacent_vertices)
        new_z = (1 - n * beta) * vertex.z + beta * sum(v.z for v in adjacent_vertices)

        vertex.x = new_x
        vertex.y = new_y
        vertex.z = new_z
    else:
        boundary_edges = []
        halfedge = vertex.halfedge
        start = halfedge

        while True:
            if not halfedge.opposite:
                boundary_edges.append(halfedge)
            halfedge = halfedge.next
            if halfedge == start:
                break

        if len(boundary_edges) == 2:
            v1 = boundary_edges[0].next.vertex
            v2 = boundary_edges[1].next.vertex

            new_x = (1 / 8) * (v1.x + v2.x) + (3 / 4) * vertex.x
            new_y = (1 / 8) * (v1.y + v2.y) + (3 / 4) * vertex.y
            new_z = (1 / 8) * (v1.z + v2.z) + (3 / 4) * vertex.z

            vertex.x = new_x
            vertex.y = new_y
            vertex.z = new_z

def update_all_old_vertices(mesh):
    for vertex in mesh.vertices:
        if vertex.index not in new_vertices:  # Only update old vertices
            update_old_vertex_position(vertex)
def contains_same_elements(arr1, arr2):
        return set(arr1) == set(arr2)
def create_new_vertex_realation_to_halfedge(mesh, new_vertex_prop):
    facet_vertices = [] 
    processed_facets = set()  
    all_facets = set() 
    
    for prop in new_vertex_prop:
        all_facets.add(prop['facet'].index)
        all_facets.add(prop['opposite_facet'].index)
    for prop in new_vertex_prop:
        for f in all_facets:
            if f not in processed_facets:
                vertices = set()
                for p in new_vertex_prop:
                    if p['halfedge'].facet.index == f:
                        vertices.add((p['vertex'], p['halfedge'].index))
                    if p['halfedge'].opposite.facet.index == f and p['halfedge']:
                        vertices.add((p['vertex'], p['halfedge'].opposite.index))
                facet_vertices.append((f, list(vertices)))
                processed_facets.add(f)  

   
    # total_vertices = 0
    # for f, vertices in facet_vertices:
    #     print(f"Facet {f} has {len(vertices)} vertices: {vertices}")
    #     total_vertices += len(vertices)
    # print(f"Total vertices: {total_vertices}")
    return facet_vertices

def create_new_vertex_realation_to_halfedge_non_shared(mesh, new_vertex_prop):
    facet_vertices = [] 
    processed_facets = set()  
    all_facets = set() 
    
    for prop in new_vertex_prop:
        all_facets.add(prop['facet'].index)
    for prop in new_vertex_prop:
        for f in all_facets:
            if f not in processed_facets:
                vertices = set()
                for p in new_vertex_prop:
                    if p['halfedge'].facet.index == f:
                        vertices.add((p['vertex'], p['halfedge'].index))
                facet_vertices.append((f, list(vertices)))
                processed_facets.add(f)  

   
    # total_vertices = 0
    # for f, vertices in facet_vertices:
    #     print(f"Facet {f} has {len(vertices)} vertices: {vertices}")
    #     total_vertices += len(vertices)
    # print(f"Total vertices: {total_vertices}")
    return facet_vertices

def update_all_faces(mesh, facet_vertices):
    faces_info = []  
    vertices_info = []  

    
    for halfedge in mesh.halfedges:
        halfedge_index = halfedge.index
        prev_halfedge_index = halfedge.next.index
        
        
        new_vertex = next((v for f, vertices in facet_vertices for v, he_index in vertices if he_index == halfedge_index), None)
        prev_new_vertex = next((v for f, vertices in facet_vertices for v, he_index in vertices if he_index == prev_halfedge_index), None)
        if new_vertex:
            original_vertex = halfedge.vertex
            vertices_array = [original_vertex.index, new_vertex.index, prev_new_vertex.index]
            faces_info.append(vertices_array)  

        next_halfedge = halfedge.next
        next_next_halfedge = halfedge.next.next
        next_new_vertex = next((v for f, vertices in facet_vertices for v, he_index in vertices if he_index == next_halfedge.index), None)
        next_next_new_vertex = next((v for f, vertices in facet_vertices for v, he_index in vertices if he_index == next_next_halfedge.index), None)
        new_vertices_array=[new_vertex.index,next_next_new_vertex.index,next_new_vertex.index]
        
        if not any(contains_same_elements(new_vertices_array, face) for face in faces_info):
            faces_info.append(new_vertices_array) 




    for vertex in mesh.vertices:
        vertex_info = (vertex.index, vertex.x, vertex.y, vertex.z)
        vertices_info.append(vertex_info) 




    # print("Faces Info:")
    # for face in faces_info:
    #     print(face)

    # print("\nVertices Info:")
    # for vertex in vertices_info:
    #     print(vertex)
    
    return faces_info, vertices_info
    
def make_off_file(vertices_info, faces_info, file_name='output.off'):
    with open(file_name, 'w') as open_file:
        
        open_file.write("OFF\n")
        open_file.write(f"{len(vertices_info)} {len(faces_info)} 0\n")
        
        
        for vertex in vertices_info:
            index, x, y, z = vertex
            open_file.write(f"{x} {y} {z}\n")
        
        
        for face in faces_info:
            v1, v2, v3 = face
            open_file.write(f"3 {v3} {v2} {v1}\n")


   

   
#opposite
    # for f, vertices in facet_vertices:
    #     print(f"Facet {f} contains: ")
    #     for v, halfedge_index in vertices:
    #        print(f"  Vertex {v.index} is part of halfedge {halfedge_index}")
        
# Assign the function to the mesh object
mesh.shared_edge_new_vertex = shared_edge_new_vertex
mesh.non_shared_edge_new_vertex = non_shared_edge_new_vertex
mesh.update_all_old_vertices=update_all_old_vertices
mesh.create_new_vertex_realation_to_halfedge=create_new_vertex_realation_to_halfedge
mesh.update_all_faces=update_all_faces
mesh.print_all_facets_vertices=print_all_facets_vertices
mesh.make_off_file=make_off_file
mesh.create_new_vertex_realation_to_halfedge_non_shared=create_new_vertex_realation_to_halfedge_non_shared


new_vertex_properties_shared = mesh.shared_edge_new_vertex(mesh)
new_vertex_properties_non_shared = mesh.non_shared_edge_new_vertex(mesh)


mesh.update_all_old_vertices(mesh)
# 合并 shared 和 non-shared 的 facet_vertices
facet_vertices_shared = mesh.create_new_vertex_realation_to_halfedge(mesh, new_vertex_properties_shared)
facet_vertices_non_shared = mesh.create_new_vertex_realation_to_halfedge_non_shared(mesh, new_vertex_properties_non_shared)

# 创建一个字典来存储合并后的 facet 和顶点关系
facet_vertices_dict = {}

# 处理 shared 的 facet_vertices
for f, vertices in facet_vertices_shared:
    if f not in facet_vertices_dict:
        facet_vertices_dict[f] = set(vertices)
    else:
        facet_vertices_dict[f].update(vertices)

# 处理 non-shared 的 facet_vertices
for f, vertices in facet_vertices_non_shared:
    if f not in facet_vertices_dict:
        facet_vertices_dict[f] = set(vertices)
    else:
        facet_vertices_dict[f].update(vertices)

# # 打印 facet_vertices_dict
# print("Facet Vertices Dictionary:")
# for f, vertices in facet_vertices_dict.items():
#     print(f"Facet {f}: Vertices {list(vertices)}")

# 转换回列表格式
facet_vertices_combined = [(f, list(vertices)) for f, vertices in facet_vertices_dict.items()]



faces_info, vertices_info = mesh.update_all_faces(mesh, facet_vertices_combined)
make_off_file(vertices_info, faces_info)



{'OFF': <bound method HalfedgeMesh.parse_off of <mesh.HalfedgeMesh object at 0x10718ad90>>}
[252, 500, 0]


In [18]:
''' read the file line by line to understanding how the half edge load 
an off file and to save the half edge mesh as an obj file.

author:
    zhangsihao yang
'''


# data_path = 'tests/data/teapot.off'
data_path = 'output.off'

import mesh

# HalfedgeMesh
mesh = mesh.HalfedgeMesh(data_path)

# Returns a list of Vertex type (in order of file)--similarly for halfedges,
# and facets
mesh.vertices

# The number of facets in the mesh


# Get the 10th halfedge


# Get the halfedge that starts at vertex 25 and ends at vertex 50



# to save the halfedge mesh you will need to following function.
def save_halfmesh_as_obj(mesh, file_name):
    with open(file_name, 'w') as open_file:
        for vertex in mesh.vertices:
            open_file.write("v {} {} {} \n".format(vertex.x, vertex.y, vertex.z))

        for face in mesh.facets:
            open_file.write("f {} {} {}\n".format(face.halfedge.vertex.index+1, face.halfedge.next.vertex.index+1, face.halfedge.next.next.vertex.index+1))

save_halfmesh_as_obj(mesh, 'first_iteration.obj')


{'OFF': <bound method HalfedgeMesh.parse_off of <mesh.HalfedgeMesh object at 0x1210b3730>>}
[1002, 2000, 0]


In [19]:
''' read the file line by line to understanding how the half edge load 
an off file and to save the half edge mesh as an obj file.

author:
    zhangsihao yang
'''
# data_path = 'tests/data/teapot.off'
data_path = 'output.off'


import mesh
import math
from mesh import Halfedge
from mesh import Vertex
# HalfedgeMesh
mesh = mesh.HalfedgeMesh(data_path)
new_vertices = set()

# Assuming Vertex class and other related structures are defined in mesh.py
def print_all_facets_vertices(mesh):
    for facet in mesh.facets:
        vertices = set()
        start_halfedge = facet.halfedge
        current_halfedge = start_halfedge

        while True:
            vertices.add(current_halfedge.vertex)
            current_halfedge = current_halfedge.next
            if current_halfedge == start_halfedge:
                break

        
        vertex_indices = [v.index for v in vertices]
        print(f"Vertices in facet {facet.index}: {vertex_indices}")

def generate_vertex_at_midpoint(mesh, v0, v1, v2, v3):
    new_x = (3/8) * (v0.x + v2.x) + (1/8) * (v1.x + v3.x)
    new_y = (3/8) * (v0.y + v2.y) + (1/8) * (v1.y + v3.y)
    new_z = (3/8) * (v0.z + v2.z) + (1/8) * (v1.z + v3.z)

    new_vertex = Vertex(
        x=new_x,
        y=new_y,
        z=new_z,
        index=len(mesh.vertices),
    )
    mesh.vertices.append(new_vertex)
    new_vertices.add(new_vertex.index)
    return new_vertex

def shared_edge_new_vertex(mesh):
    printed_edges = set()
    new_vertex_prop = []
    for shared_halfedge in mesh.halfedges:
        if shared_halfedge.opposite:
            edge = tuple(sorted([shared_halfedge.vertex.index, shared_halfedge.opposite.vertex.index]))
            if edge not in printed_edges:
                printed_edges.add(edge)
                v0 = shared_halfedge.vertex.index
                v2 = shared_halfedge.opposite.vertex.index
                f1 = shared_halfedge.facet.index
                f2 = shared_halfedge.opposite.facet.index
                # print(f"Shared edge between vertices {v0} and {v2} is shared by facets {f1} and {f2}")
                
                # Get the four vertices of the two facets
                f1_vertices = [
                    shared_halfedge.vertex,
                    shared_halfedge.next.vertex,
                    shared_halfedge.next.next.vertex
                ]
                f2_vertices = [
                    shared_halfedge.opposite.vertex,
                    shared_halfedge.opposite.next.vertex,
                    shared_halfedge.opposite.next.next.vertex
                ]

                # Combine and remove duplicates
                all_vertices = list(set(f1_vertices + f2_vertices))
                
                # Print the unique vertices
                # print(f"Facet {f1} and Facet {f2} unique vertices: {[v.index for v in all_vertices]}")

                # Ensure we have exactly four unique vertices
                if len(all_vertices) == 4:
                    # Ensure v0 and v2 are the vertices of the shared edge
                    v0 = shared_halfedge.vertex
                    v2 = shared_halfedge.opposite.vertex

                    # The other two vertices
                    remaining_vertices = [v for v in all_vertices if v != v0 and v != v2]
                    v1, v3 = remaining_vertices
                    
                    new_vertex = generate_vertex_at_midpoint(mesh, v0, v1, v2, v3)

                    # Store the new vertex properties
                    new_vertex_prop.append({
                        'vertex': new_vertex,
                        'halfedge': shared_halfedge,
                        'facet': shared_halfedge.facet,
                        'opposite_facet':shared_halfedge.opposite.facet,
                        'start': shared_halfedge.vertex.index,
                        'end': shared_halfedge.opposite.vertex.index
                    })

                    # for prop in new_vertex_prop:
                    #    print(f"Shared new vertex at ({prop['vertex'].x}, {prop['vertex'].y}, {prop['vertex'].z}, start {prop['start']}, end {prop['end']}) on halfedge {prop['halfedge'].index} in facet {prop['facet'].index}")
    return new_vertex_prop

def generate_vertex_at_midpoint_non_shared(mesh, v1, v2):
    new_x = (1/2) * (v1.x + v2.x)
    new_y = (1/2) * (v1.y + v2.y)
    new_z = (1/2) * (v1.z + v2.z)

    new_vertex = Vertex(
        x=new_x,
        y=new_y,
        z=new_z,
        index=len(mesh.vertices),
    )
    mesh.vertices.append(new_vertex)
    new_vertices.add(new_vertex.index)
    return new_vertex

def non_shared_edge_new_vertex(mesh):
    printed_edges = set()
    new_vertex_prop = []
    
    for halfedge in mesh.halfedges:
        if not halfedge.opposite:
            edge = tuple(sorted([halfedge.vertex.index, halfedge.next.vertex.index]))
            if edge not in printed_edges:
                printed_edges.add(edge)
                v1 = halfedge.vertex
                v2 = halfedge.prev.vertex
                # print(f"Non-shared edge between vertices {v1.index} and {v2.index}")
                
                # Generate the new vertex at the midpoint
                new_vertex = generate_vertex_at_midpoint_non_shared(mesh, v1, v2)

                # Store the new vertex properties
                new_vertex_prop.append({
                    'vertex': new_vertex,
                    'halfedge': halfedge,
                    'facet': halfedge.facet,
                    'start': halfedge.vertex.index
                })
    # Print new vertex properties
    # for prop in new_vertex_prop:
    #     print(f"Non-shared new vertex at ({prop['vertex'].x}, {prop['vertex'].y}, {prop['vertex'].z}) on halfedge {prop['halfedge'].index} in facet {prop['facet'].index}")

    return new_vertex_prop

def calculate_beta(n):
    if n == 3:
        return 3 / 16
    else:
        return (1 / n) * (5 / 8 - (3 / 8 + 1 / 4 * math.cos(2 * math.pi / n)) ** 2)

def update_old_vertex_position(vertex):
    adjacent_vertices = []
    halfedge = vertex.halfedge
    start = halfedge

    if halfedge.opposite:
        adjacent_vertices = []
        halfedge = vertex.halfedge
        start = halfedge

        while True:
            if halfedge.opposite:  # Ensure opposite is not None
                adjacent_vertices.append(halfedge.opposite.vertex)
                halfedge = halfedge.opposite.prev
            else:
                break
            if halfedge == start:
                break

        n = len(adjacent_vertices)
        beta = calculate_beta(n)

        new_x = (1 - n * beta) * vertex.x + beta * sum(v.x for v in adjacent_vertices)
        new_y = (1 - n * beta) * vertex.y + beta * sum(v.y for v in adjacent_vertices)
        new_z = (1 - n * beta) * vertex.z + beta * sum(v.z for v in adjacent_vertices)

        vertex.x = new_x
        vertex.y = new_y
        vertex.z = new_z
    else:
        boundary_edges = []
        halfedge = vertex.halfedge
        start = halfedge

        while True:
            if not halfedge.opposite:
                boundary_edges.append(halfedge)
            halfedge = halfedge.next
            if halfedge == start:
                break

        if len(boundary_edges) == 2:
            v1 = boundary_edges[0].next.vertex
            v2 = boundary_edges[1].next.vertex

            new_x = (1 / 8) * (v1.x + v2.x) + (3 / 4) * vertex.x
            new_y = (1 / 8) * (v1.y + v2.y) + (3 / 4) * vertex.y
            new_z = (1 / 8) * (v1.z + v2.z) + (3 / 4) * vertex.z

            vertex.x = new_x
            vertex.y = new_y
            vertex.z = new_z

def update_all_old_vertices(mesh):
    for vertex in mesh.vertices:
        if vertex.index not in new_vertices:  # Only update old vertices
            update_old_vertex_position(vertex)
def contains_same_elements(arr1, arr2):
        return set(arr1) == set(arr2)
def create_new_vertex_realation_to_halfedge(mesh, new_vertex_prop):
    facet_vertices = [] 
    processed_facets = set()  
    all_facets = set() 
    
    for prop in new_vertex_prop:
        all_facets.add(prop['facet'].index)
        all_facets.add(prop['opposite_facet'].index)
    for prop in new_vertex_prop:
        for f in all_facets:
            if f not in processed_facets:
                vertices = set()
                for p in new_vertex_prop:
                    if p['halfedge'].facet.index == f:
                        vertices.add((p['vertex'], p['halfedge'].index))
                    if p['halfedge'].opposite.facet.index == f and p['halfedge']:
                        vertices.add((p['vertex'], p['halfedge'].opposite.index))
                facet_vertices.append((f, list(vertices)))
                processed_facets.add(f)  

   
    # total_vertices = 0
    # for f, vertices in facet_vertices:
    #     print(f"Facet {f} has {len(vertices)} vertices: {vertices}")
    #     total_vertices += len(vertices)
    # print(f"Total vertices: {total_vertices}")
    return facet_vertices

def create_new_vertex_realation_to_halfedge_non_shared(mesh, new_vertex_prop):
    facet_vertices = [] 
    processed_facets = set()  
    all_facets = set() 
    
    for prop in new_vertex_prop:
        all_facets.add(prop['facet'].index)
    for prop in new_vertex_prop:
        for f in all_facets:
            if f not in processed_facets:
                vertices = set()
                for p in new_vertex_prop:
                    if p['halfedge'].facet.index == f:
                        vertices.add((p['vertex'], p['halfedge'].index))
                facet_vertices.append((f, list(vertices)))
                processed_facets.add(f)  

   
    # total_vertices = 0
    # for f, vertices in facet_vertices:
    #     print(f"Facet {f} has {len(vertices)} vertices: {vertices}")
    #     total_vertices += len(vertices)
    # print(f"Total vertices: {total_vertices}")
    return facet_vertices

def update_all_faces(mesh, facet_vertices):
    faces_info = []  
    vertices_info = []  

    
    for halfedge in mesh.halfedges:
        halfedge_index = halfedge.index
        prev_halfedge_index = halfedge.next.index
        
        
        new_vertex = next((v for f, vertices in facet_vertices for v, he_index in vertices if he_index == halfedge_index), None)
        prev_new_vertex = next((v for f, vertices in facet_vertices for v, he_index in vertices if he_index == prev_halfedge_index), None)
        if new_vertex:
            original_vertex = halfedge.vertex
            vertices_array = [original_vertex.index, new_vertex.index, prev_new_vertex.index]
            faces_info.append(vertices_array)  

        next_halfedge = halfedge.next
        next_next_halfedge = halfedge.next.next
        next_new_vertex = next((v for f, vertices in facet_vertices for v, he_index in vertices if he_index == next_halfedge.index), None)
        next_next_new_vertex = next((v for f, vertices in facet_vertices for v, he_index in vertices if he_index == next_next_halfedge.index), None)
        new_vertices_array=[new_vertex.index,next_next_new_vertex.index,next_new_vertex.index]
        
        if not any(contains_same_elements(new_vertices_array, face) for face in faces_info):
            faces_info.append(new_vertices_array) 




    for vertex in mesh.vertices:
        vertex_info = (vertex.index, vertex.x, vertex.y, vertex.z)
        vertices_info.append(vertex_info) 




    # print("Faces Info:")
    # for face in faces_info:
    #     print(face)

    # print("\nVertices Info:")
    # for vertex in vertices_info:
    #     print(vertex)
    
    return faces_info, vertices_info
    
def make_off_file(vertices_info, faces_info, file_name='output.off'):
    with open(file_name, 'w') as open_file:
        
        open_file.write("OFF\n")
        open_file.write(f"{len(vertices_info)} {len(faces_info)} 0\n")
        
        
        for vertex in vertices_info:
            index, x, y, z = vertex
            open_file.write(f"{x} {y} {z}\n")
        
        
        for face in faces_info:
            v1, v2, v3 = face
            open_file.write(f"3 {v3} {v2} {v1}\n")


   

   
#opposite
    # for f, vertices in facet_vertices:
    #     print(f"Facet {f} contains: ")
    #     for v, halfedge_index in vertices:
    #        print(f"  Vertex {v.index} is part of halfedge {halfedge_index}")
        










# Assign the function to the mesh object
mesh.shared_edge_new_vertex = shared_edge_new_vertex
mesh.non_shared_edge_new_vertex = non_shared_edge_new_vertex
mesh.update_all_old_vertices=update_all_old_vertices
mesh.create_new_vertex_realation_to_halfedge=create_new_vertex_realation_to_halfedge
mesh.update_all_faces=update_all_faces
mesh.print_all_facets_vertices=print_all_facets_vertices
mesh.make_off_file=make_off_file
mesh.create_new_vertex_realation_to_halfedge_non_shared=create_new_vertex_realation_to_halfedge_non_shared


new_vertex_properties_shared = mesh.shared_edge_new_vertex(mesh)
new_vertex_properties_non_shared = mesh.non_shared_edge_new_vertex(mesh)


mesh.update_all_old_vertices(mesh)
# 合并 shared 和 non-shared 的 facet_vertices
facet_vertices_shared = mesh.create_new_vertex_realation_to_halfedge(mesh, new_vertex_properties_shared)
facet_vertices_non_shared = mesh.create_new_vertex_realation_to_halfedge_non_shared(mesh, new_vertex_properties_non_shared)

# 创建一个字典来存储合并后的 facet 和顶点关系
facet_vertices_dict = {}

# 处理 shared 的 facet_vertices
for f, vertices in facet_vertices_shared:
    if f not in facet_vertices_dict:
        facet_vertices_dict[f] = set(vertices)
    else:
        facet_vertices_dict[f].update(vertices)

# 处理 non-shared 的 facet_vertices
for f, vertices in facet_vertices_non_shared:
    if f not in facet_vertices_dict:
        facet_vertices_dict[f] = set(vertices)
    else:
        facet_vertices_dict[f].update(vertices)

# # 打印 facet_vertices_dict
# print("Facet Vertices Dictionary:")
# for f, vertices in facet_vertices_dict.items():
#     print(f"Facet {f}: Vertices {list(vertices)}")

# 转换回列表格式
facet_vertices_combined = [(f, list(vertices)) for f, vertices in facet_vertices_dict.items()]



# 使用合并后的 facet_vertices 进行后续处理
faces_info, vertices_info = mesh.update_all_faces(mesh, facet_vertices_combined)
make_off_file(vertices_info, faces_info)






{'OFF': <bound method HalfedgeMesh.parse_off of <mesh.HalfedgeMesh object at 0x121067d90>>}
[1002, 2000, 0]


In [20]:
''' read the file line by line to understanding how the half edge load 
an off file and to save the half edge mesh as an obj file.

author:
    zhangsihao yang
'''
print ('hello')

# data_path = 'tests/data/teapot.off'
data_path = 'output.off'

import mesh

# HalfedgeMesh
mesh = mesh.HalfedgeMesh(data_path)
print(mesh.halfedges[2].vertex.index)
print(mesh.halfedges[2].prev.vertex.index)

# Returns a list of Vertex type (in order of file)--similarly for halfedges,
# and facets
mesh.vertices

# The number of facets in the mesh
print(len(mesh.facets))

# Get the 10th halfedge


# Get the halfedge that starts at vertex 25 and ends at vertex 50


print(mesh.vertices)
for vertex in mesh.vertices:
    print(vertex.get_vertex())

print(mesh.facets)
print('--------------------')
for face in mesh.facets:
    print(face.a, face.b, face.c)

# to save the halfedge mesh you will need to following function.
def save_halfmesh_as_obj(mesh, file_name):
    with open(file_name, 'w') as open_file:
        for vertex in mesh.vertices:
            open_file.write("v {} {} {} \n".format(vertex.x, vertex.y, vertex.z))

        for face in mesh.facets:
            open_file.write("f {} {} {}\n".format(face.halfedge.vertex.index+1, face.halfedge.next.vertex.index+1, face.halfedge.next.next.vertex.index+1))

save_halfmesh_as_obj(mesh, 'second_iteration.obj')


hello
{'OFF': <bound method HalfedgeMesh.parse_off of <mesh.HalfedgeMesh object at 0x1075fde80>>}
[4002, 8000, 0]
1003
252
8000
[<mesh.Vertex object at 0x1075fd880>, <mesh.Vertex object at 0x1075fdb20>, <mesh.Vertex object at 0x1075fdac0>, <mesh.Vertex object at 0x1075fd850>, <mesh.Vertex object at 0x1075fd7c0>, <mesh.Vertex object at 0x1075fd760>, <mesh.Vertex object at 0x1075fd820>, <mesh.Vertex object at 0x1075e1eb0>, <mesh.Vertex object at 0x1075e1f10>, <mesh.Vertex object at 0x1075e1fa0>, <mesh.Vertex object at 0x1075e1ee0>, <mesh.Vertex object at 0x1075e1d60>, <mesh.Vertex object at 0x1075e18b0>, <mesh.Vertex object at 0x1213765b0>, <mesh.Vertex object at 0x121376f70>, <mesh.Vertex object at 0x121376400>, <mesh.Vertex object at 0x1075fe520>, <mesh.Vertex object at 0x1075fe550>, <mesh.Vertex object at 0x1075fe580>, <mesh.Vertex object at 0x1075fe610>, <mesh.Vertex object at 0x1075fe6d0>, <mesh.Vertex object at 0x1075fe790>, <mesh.Vertex object at 0x1075fe7c0>, <mesh.Vertex object 